# Using the GIS object

Import GIS from the `arcgis.gis` module:

In [ ]:
from arcgis.gis import GIS

To create a GIS object, pass the url and login credentials:

In [ ]:
import getpass

online_username = input('Username: ')
online_password = getpass.getpass('Password: ')
online_connection = "https://www.arcgis.com"


gis = GIS(online_connection, 
          online_username, 
          online_password)

Adding a '?' mark after an object and querying it brings up help for that object in the notebook:

In [ ]:
gis?

To access a user, use the `user` property of your `gis` object, which gives you an instance of the `UserManager` class.  Then call the `get()` method of the `UserManager` object to pass the user name of the user you are interested in.

In [ ]:
user = gis.users.get('gonzalezmorales_undesa')

The ArcGIS API for Python is integrated with Jupyter Notebook to make it easy to visualize and interact with GIS resources. The `user` object has a rich representation that can be queried like this:

In [ ]:
user

Resources are implemented as Python dictionaries.  You can query for the resource properties using the `resource['property']` notation:

In [ ]:
for property, value in vars(user).items():
    print( property)

In [ ]:
user['firstName']

The properties are also available as properties on the resource object, so you can use the dot notation to access them:

In [ ]:
user.lastName

# Accessing and creating content
Using the `gis` module, you can search for, access, and manage all your folders, contents, and automate such maintenance workflows as scripts

## Search for content in your GIS
- Accessing the `content` property of your `gis` object returns an instance of the `ContentManager` class. 
- You can use the `search` method of the `ContentManager` object to search for items in your GIS
- The `query` parameter supports a variety of inputs (see [search reference](http://resources.arcgis.com/en/help/arcgis-rest-api/#/Search_reference/02r3000000mn000000/)).


### Example 1:
Access the `ContentManager` through the `content` property of your `GIS` object to search for items of type _"feature layer"_ that have _"international poverty line"_ in their title. 

For each feature layer that gets returned, display its rich representation within the notebook:

In [ ]:
from IPython.display import display

items = gis.content.search(query='title:international poverty line', 
                           item_type='Feature Layer')
for item in items:
    display(item)

### Example 2:

Limit the search to only your own content (excluding items from other users that are shared across the organization) by specifying the owner in the query string

In [ ]:
my_items = gis.content.search(query="owner:unstats_admin", item_type="csv", max_items=800)

for item in my_items[0:10]:
    print(item)
    display(item)

### Example 3:
Use one asterisk * for multiple character wild card search, and one question mark ? for single character:

In [ ]:
# search for content that begin with a prefix - say 'unstats'
search_result_unstats = gis.content.search(query='owner:unstats*')
search_result_unstats

### Example 4:
Once you know an item's id (called itemId), you can access it using the get() method instead of searching for it.

In [ ]:
# lets get the itemid of first item from previous query
first_item = search_result_unstats[0]
known_item_id = first_item.id
print(known_item_id)

In [ ]:
# Use the get() to access this item
online_banking_item = gis.content.get(known_item_id)
online_banking_item

## Search for content published by users outside your organization

To search for content published by other named users and shared publicly, turn the `outside_org` flag to `True`:

In [ ]:
sdgs = gis.content.search(query="lao", item_type = "web scene",
                                           sort_field="numViews" ,sort_order="asc",
                                           max_items = 15, outside_org=True)
for item in sdgs:
    display(item)

## Create new content
Use the `add()` method on the `ContentManager` object (which is accessed through the `content` property of your `GIS` object)
- The `add()` method accepts a dictionary containing the properties of the item being created
- The optional `data` parameter accepts a string path to a file
- The optional `metadata` parameter specifies an XML files containing metadata information

All the content you add this way is added to the authenticated user's content

### Example 5
Add a csv file as a csv item

In [ ]:
csv_path = r"https://raw.githubusercontent.com/UNStats/FIS4SDGs/master/unsd/data/csv/country_1.1.1-SI_POV_DAY1_wide.csv"
csv_properties={'title':'Proportion of population below international poverty line (percent)',
                'description':'Goal 1: End poverty in all its forms everywhere <br> TargetCode 1.1: By 2030, eradicate extreme poverty for all people everywhere, currently measured as people living on less than $1.25 a day <br> Indicator 1.1.1: Proportion of population below the international poverty line, by sex, age, employment status and geographical location (urban/rural)<br>Series: SI_POV_DAY1 - Proportion of population below international poverty line (percent)',
                'tags':'poverty line, poverty, standard of living, basic needs'}
thumbnail_path = r"https://raw.githubusercontent.com/UNStats/FIS4SDGs/master/globalResources/sdgIcons120x120/SDG01.jpg"

test_csv_item = gis.content.add(item_properties=csv_properties, 
                                data=csv_path,
                                thumbnail = thumbnail_path)

The `add()` method returns an object of type arcgis.gis.Item representing the added item. In Jupyter notebook environment, you can visualize it by querying the item:

In [ ]:
test_csv_item

*Note*: You can specify most properties of an item as a dictionary to the `item_properties` parameter. Refer to the [API ref doc ](http://esri.github.io/arcgis-python-api/apidoc/html/arcgis.gis.html#arcgis.gis.ContentManager.add) of this method for a definitive list. 

You could also specify an empty dictionary to this parameter, the downside is search cannot index these items efficiently if it does not have sufficient metadata information, hence that is not a good programming practice.

The following is a list of all the properties of the csv item just created:

In [ ]:
for property, value in vars(test_csv_item).items():
    print( property)

## Publishing an item as a web layer
Call the `publish()` method on them following types of items to publish them into hosted web layers:
- csv
- service definitions
- file geodatabases
- shape files
- packages such as tile, scene and vector tile 

The `publish()` method accepts an optional `publish_parameters` dictionary specifying certain parameters supported by the publish REST API

The `address_fields` parameter allows to specify which field in a csv or dataset contains address fields suitable for geocoding.


### Example 6
Publish the csv from the previous example into a web feature layer. 

This csv contains latitude and longitude coordinates of each point, hence the `address_fields` parameter can be ignored. 
The `publish()` method returns an `Item` object corresponding to the web layer created.

In [ ]:
test_feature_layer_item2 = test_csv_item.publish()

In [ ]:
test_feature_layer_item2

Notice that the GIS used the same metadata (thumbnail, title, description) of the source item used for publishing. This helps provide context on what the new item is and also as a good starting point for the metadata which you can edit later.

## Importing data from a pandas data frame

- Pandas is a popular data analysis library for Python. 
- The data frame object of pandas allows to store and analyze tabular information. 
- The `import_data() method` allows to import pandas dataframes as an `arcgis.features.FeatureCollection` object 
- The feature collection can then be added as an `item` to your `GIS`.

### Example 7
Create a new pandas data frame from a csv file, filter that data through pandas, and add it as a feature collection.

In [ ]:
# read csv as a pandas dataframe
import pandas

SI_POV_DAY1_df = pandas.read_csv(r'https://raw.githubusercontent.com/UNStats/FIS4SDGs/master/unsd/data/csv/1.1.1-SI_POV_DAY1_long.csv',encoding ='latin1')
SI_POV_DAY1_df

List the name of all columns in the data frame:

In [ ]:
list(SI_POV_DAY1_df)

In [ ]:
# find Value of Ethiopia after 1999
ethiopia = SI_POV_DAY1_df.loc[(SI_POV_DAY1_df.GeoArea_Desc == 'Ethiopia') & (SI_POV_DAY1_df.Year > 1999)][['Year','Value']]
ethiopia

In [ ]:
# only select values for years after 2009:
SI_POV_DAY1_2015 = SI_POV_DAY1_df.loc[(SI_POV_DAY1_df.Year == 2015)]
SI_POV_DAY1_2015

Import the data frame as a feature collection:

In [ ]:
SI_POV_DAY1_2015_fc = gis.content.import_data(SI_POV_DAY1_2015)
SI_POV_DAY1_2015_fc

Convert the feature collection to JSON and add it as a text based item to the GIS:

In [ ]:
import json
SI_POV_DAY1_2015_fc_dict = dict(SI_POV_DAY1_2015_fc.properties)
SI_POV_DAY1_2015_json = json.dumps({"featureCollection": {"layers": [SI_POV_DAY1_2015_fc_dict]}})

In [ ]:
SI_POV_DAY1_2015_item_properties = {'title': 'Proportion of population below international poverty line',
                        'description':'Example demonstrating conversion of pandas ' + \
                         'dataframe object to a GIS item',
                        'tags': 'arcgis python api, pandas, csv',
                        'text':SI_POV_DAY1_2015_json,
                        'type':'Feature Collection'}

SI_POV_DAY1_2015_item = gis.content.add(SI_POV_DAY1_2015_item_properties)
SI_POV_DAY1_2015_item

**Note**: The `add()` method was used to add the `FeatureCollection` object in memory as an `item` on the `GIS`. Notice that the `text` property was used to pass the JSON representation of the feature collection, and the `type` property to indicate the item type.  You can use the same method to publish web maps and web scenes.

## Publishing an empty service
The `ContentManager` class allows to publish a new service without any content, using the `create_service()` method (e.g., in order to add content later by editing the servcie from client applications).

Before creating a service, check whether the desired unique service name is available using the `is_service_name_available()` method.

In [ ]:
# check if service name is available
gis.content.is_service_name_available(service_name= "awesome_python", service_type = 'featureService')

In [ ]:
# let us publish an empty service
empty_service_item = gis.content.create_service(name='awesome_python', service_type='featureService')
empty_service_item

In [ ]:
# access the layers property of the item
empty_service_item.layers

# Organizing content

The `create_folder()` and `delete_folder()` methods available on the `ContentManager` class can be used to manage folders. 

Once folders are created, the `move()` method is used to move content into and out them.

### Example 8
Create a folder called 'tests' and move the `SI_POV_DAY1_2015_item` into it

In [ ]:
# create new folder
gis.content.create_folder(folder= 'tests')

In [ ]:
# move the SI_POV_DAY1_2015_item into the tests folder
SI_POV_DAY1_2015_item.move(folder= 'tests')

In [ ]:
# move the empty_service_item into the tests folder
empty_service_item.move(folder= 'tests')

To move content out of folders to root, call the `move()` method and specify `/` as folder name

In [ ]:
# move back to root
empty_service_item.move(folder='/')

# Managing content

As an organization matures and expands its GIS, users add items of various types and properties with varying relationships to one another. 

This section demonstrates how to 
- retrieve item properties
- delete an existing item
- examine relationships between items

## Access an item

In [ ]:
#access an Item
SI_POV_DAY1_2015_item = gis.content.get('f4b8621d0c9b466d8f3222e448c5329d')
SI_POV_DAY1_2015_item

In [ ]:
# item id
SI_POV_DAY1_2015_item.id

In [ ]:
# title
SI_POV_DAY1_2015_item.title

In [ ]:
# tags
SI_POV_DAY1_2015_item.tags

## Updating item properties
You can update any of the Items properties using the `update()` method. It accepts parameters similar to `add()` method.

In [ ]:
# update the tags
SI_POV_DAY1_2015_item.update(item_properties={'tags':'poverty, test, new Tag'})

In [ ]:
# updating thumbnail
SI_POV_DAY1_2015_item.update(thumbnail= r'https://raw.githubusercontent.com/UNStats/FIS4SDGs/master/globalResources/sdgIcons120x120/SDG01.jpg')

In [ ]:
SI_POV_DAY1_2015_item

## Downloading items
Downloading components of your items, such as the thumbnail, data, or metadata, is helpful in order to archive content or migrate content.

### Download item data
The `get_data()` function reads the binary or text data associated with an item and returns it as a byte array.
- If the text data is JSON, it converts the data to a Python dictionary. If it is another text format, it returns the file as a string.
- If data is not text, binary files are returned along with the path to where the data is downloaded.

In [ ]:
ports_csv_item = gis.content.get('a184c6a497f848f098f3960ec94f7443')
ports_csv_item

The data for a csv item is the csv file itself. It's downloaded to your default temporary directory

In [ ]:
ports_csv_item.get_data()

## Download metadata
Metadata can be downloaded into an XML file using the `download_metadata()` method. Once we have the XML file we can edit it and then modify existing portal items by entering the file name as the metadata parameter of the Item.update() method. You can also add it with the metadata parameter on the ContentManager.add() method when creating new content. See the specific documentation for enabling metadata with ArcGIS Online or ArcGIS Enterprise.